<a href="https://colab.research.google.com/github/zeon62/TinyML/blob/main/struct4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

sine wave example 기반으로 여러 모델 형태에 대한 구조를 파악하기 위한 모델 출력 코드

In [ ]:
# tensorflow 설치 // 23.3 기준 최신버전
!pip install tensorflow==2.11.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import numpy as np
import math
import os

In [ ]:
# Set input value - sine wave
SAMPLES = 1000
SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)

x_values = np.random.uniform(low=0, high=2*math.pi, size=SAMPLES)
np.random.shuffle(x_values)
y_values = np.sin(x_values)
y_values += 0.1 * np.random.randn(*y_values.shape)

TRAIN_SPLIT = int(0.6*SAMPLES)
TEST_SPLIT = int(0.2*SAMPLES + TRAIN_SPLIT)

x_train, x_validate, x_test = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
y_train, y_validate, y_test = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

In [ ]:
# Build Model - save (build)
from tensorflow.keras import layers
model = tf.keras.Sequential()
model.add(layers.Dense(4, activation = 'relu', input_shape=(1,)))
model.add(layers.Dense(1))

model.save("model/build/build.keras") # Native Keras format
model.save("model/build/build.tf") # TensorFlow SavedModel format
model.save("model/build/build.h5") # Legacy HDF5 format

In [ ]:
# Compile Model // compile function - save (compile)
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

model.save("model/compile/compile.keras")
model.save("model/compile/compile.tf")
model.save("model/compile/compile.h5")

In [ ]:
# Callback 함수 선언
my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='model/checkpoint/model.{epoch:02d}.ckpt')
]

In [ ]:
# Training Model // fit function - save (train)
history = model.fit(x_train, y_train, epochs=10, batch_size=1, validation_data=(x_validate, y_validate), callbacks=my_callbacks )

Epoch 1/10
600/600 [==============================] - 8s 7ms/step - loss: 0.4419 - mae: 0.5761 - val_loss: 0.3409 - val_mae: 0.5039
Epoch 2/10
600/600 [==============================] - 4s 6ms/step - loss: 0.2763 - mae: 0.4595 - val_loss: 0.2542 - val_mae: 0.4354
Epoch 3/10
600/600 [==============================] - 4s 6ms/step - loss: 0.2208 - mae: 0.4160 - val_loss: 0.2156 - val_mae: 0.4077
Epoch 4/10
600/600 [==============================] - 4s 7ms/step - loss: 0.2013 - mae: 0.3944 - val_loss: 0.2033 - val_mae: 0.3943
Epoch 5/10
600/600 [==============================] - 4s 7ms/step - loss: 0.1943 - mae: 0.3874 - val_loss: 0.1950 - val_mae: 0.3838
Epoch 6/10
600/600 [==============================] - 2s 4ms/step - loss: 0.1899 - mae: 0.3788 - val_loss: 0.1944 - val_mae: 0.3746
Epoch 7/10
600/600 [==============================] - 2s 3ms/step - loss: 0.1870 - mae: 0.3734 - val_loss: 0.1892 - val_mae: 0.3792
Epoch 8/10
600/600 [==============================] - 2s 4ms/step - loss: 0.

In [ ]:
model.save("model/train/train.keras")
model.save("model/train/train.tf")
model.save("model/train/train.h5")

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 8         
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Convert to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

open("model.tflite", "wb").write(tflite_model)

1528

In [ ]:
# Analyze Tflite Model
import sys
sys.stdout = open("model/analyzer.txt", "w")
print(tf.lite.experimental.Analyzer.analyze(model_content=tflite_model))

In [ ]:
# Convert to C array
!apt-get -qq install xxd
!xxd -i model/model.tflite > model/model.cc

In [ ]:
!zip -r model.zip model

In [ ]:
#"""
from google.colab import files
files.download("model.zip")
#"""

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>